#Training Notebook - Indian Sign Language Translator
Author : [Abhishek Singh Dhadwal](https://github.com/AbhishekSinghDhadwal)


This notebook has been used to train the squeezenet model used in our project using Transfer Learning. Originally made for training via Colab, can be run locally by changing the required DIRs.

If required, the code can be modified to transfer train via the following 4 models, you only need to download the pre-trained models and change the dir accordingly :

1. SqueezeNet (original) - [weights](https://github.com/OlafenwaMoses/ImageAI/releases/download/1.0/squeezenet_weights_tf_dim_ordering_tf_kernels.h5)
2. DenseNet - [DenseNet-BC-121-32 weights](https://github.com/OlafenwaMoses/ImageAI/releases/download/1.0/DenseNet-BC-121-32.h5)
3. ResNet 50 - [weights](https://github.com/OlafenwaMoses/ImageAI/releases/download/1.0/resnet50_weights_tf_dim_ordering_tf_kernels.h5)
4. Inceptionv3 - [weights](https://github.com/OlafenwaMoses/ImageAI/releases/download/1.0/inception_v3_weights_tf_dim_ordering_tf_kernels.h5)



### COLAB-ONLY SNIPPET
The following snippet is used for mounting Colab to Google drive, do not run if you're using a local runtime.

In [ ]:
from google.colab import drive
import subprocess

drive.mount('/content/drive')

Importing libraries-
 do note that imageAI 2.1 is not compatible with tensorflow 2, hence we rollback to a previous version for tensorflow.

In [ ]:
# Downloading imageAI library
!pip3 install imageai==2.1.5
!pip3 install -q opencv-python
!pip3 install -q pillow
!pip install tensorflow-gpu==1.15.2

### Helper functions and required imports

In [ ]:
import os
from imageai.Prediction.Custom import ModelTraining,CustomImagePrediction

def input_model(model_name) :
    model_trainer = ModelTraining()
    if model_name == "SqueezeNet":
        model_trainer.setModelTypeAsSqueezeNet()
        modeldata = "squeezenetdata"
    elif model_name == "ResNet":
        model_trainer.setModelTypeAsResNet()
        modeldata = "resnetdata"
    elif model_name ==  "Inceptionv3":
        model_trainer.setModelTypeAsInceptionV3()
        modeldata = "inceptiondata"
    elif model_name ==  "DenseNet":
        model_trainer.setModelTypeAsDenseNet()
        modeldata = "densenetdata"
    else :
        print("Invalid model name entered")
        exit(0)
    return model_trainer,modeldata

def getModelPredict(model_name, model_path, json_path, num_objects):
    model = CustomImagePrediction()
    if model_name == "SqueezeNet":
        model.setModelTypeAsSqueezeNet()
    elif model_name == "ResNet":
        model.setModelTypeAsResNet()
    elif model_name ==  "Inceptionv3":
        model.setModelTypeAsInceptionV3()
    elif model_name ==  "DenseNet":
        model.setModelTypeAsDenseNet()
    else :
        print("Invalid model name entered")
        exit(0)
    model.setModelPath(model_path)
    model.setJsonPath(json_path)
    model.loadFullModel(num_objects=num_objects)
    return model


## Inputs for pre-trained model weights and data

In [ ]:
# ENTER YOUR DIRECTORIES AND DATA HERE - This data changes per test #########################################################################
model,modeldata = input_model("SqueezeNet") # Change as per requirements 
NUM_EPOCHS = 100
model_rar_loc = "path of pre-trained model compressed in rar format"
ds_rar_loc = "path of dataset compressed in rar format"
dataset_name = "the name of your parent dataset folder before extraction" # Input for your dataset, should be in root Drive level
model_data_dir = "location where both trained models and json files should be stored"
num_classes = 10
##############################################################################################################################################

model_rar = os.path.basename(model_rar_loc)
MODEL_H5 = model_rar.replace('.rar','') + ".h5"
dataset_rar = os.path.basename(ds_rar_loc)
DATASET_NAME = dataset_rar.replace('.rar','')

MODEL_DIR=model_data_dir+"/models/"
!mkdir MODEL_DIR 
JSON_DIR= model_data_dir+"/json/"
!mkdir JSON_DIR

print("Model directory :",MODEL_DIR)
print("Json directory :",JSON_DIR)

Code for extracting RAR items to base Colab directory

In [ ]:
%%time
!apt install unrar

cpCmd = ["cp",ds_rar_loc, "."]
process = subprocess.Popen(cpCmd, stdout=subprocess.PIPE)
output, error = process.communicate()

unrarCmd = ["unrar","x","-y",dataset_rar]
process = subprocess.Popen(unrarCmd, stdout=subprocess.PIPE)
output, error = process.communicate()

cpCmd = ["cp",model_rar_loc, "."]
process = subprocess.Popen(cpCmd, stdout=subprocess.PIPE)
output, error = process.communicate()

unrarCmd = ["unrar","x","-y",model_rar]
process = subprocess.Popen(unrarCmd, stdout=subprocess.PIPE)
output, error = process.communicate()

rmCmd = ["rm",dataset_rar]
process = subprocess.Popen(rmCmd, stdout=subprocess.PIPE)
output, error = process.communicate()

rmCmd = ["rm",model_rar]
process = subprocess.Popen(unrarCmd, stdout=subprocess.PIPE)
output, error = process.communicate()

!ls

### Code for running the model
Modify enhance_data and transfer_with_full_training paramters as per your requirements

For more details, refer to the [imageAI docs](https://imageai.readthedocs.io/en/latest/custom/index.html)

In [ ]:

model.setDataDirectory(data_directory=DATASET_NAME, train_subdirectory=r"train/", test_subdirectory=r"val/",models_subdirectory=MODEL_DIR, json_subdirectory=JSON_DIR)
model.trainModel(num_objects= num_classes, num_experiments=NUM_EPOCHS, enhance_data=False, batch_size=8, transfer_from_model=MODEL_H5, transfer_with_full_training=False, show_network_summary=True, save_full_model =True)


In [ ]:
'''
%%time

# RUN ONLY IF PREVIOUS PART FAILED
last_saved_iter = r"path of the last checkpointed model h5"
epochs_left = 20
model.setDataDirectory(data_directory=DATASET_NAME train_subdirectory=r"train/", test_subdirectory=r"val/",models_subdirectory=MODEL_DIR, json_subdirectory=JSON_DIR)
model.trainModel(num_objects= num_classes, num_experiments=epochs_left, enhance_data=False, batch_size=8,transfer_from_model= last_saved_iter, transfer_with_full_training=False, show_network_summary=True, save_full_model =True)
'''

### Sample Image prediction code

In [ ]:
#Example - squeezemod = getModelPredict("SqueezeNet",MODEL_DIR+"/model_ex-041_acc-0.982818.h5",JSON_DIR+"/model_class.json",10)
squeezemod = getModelPredict("SqueezeNet",MODEL_DIR+"/model_to_be_tested",JSON_DIR+"/model_class.json",num_classes)
prediction.loadFullModel(num_objects=20)
predictions, probabilities = prediction.predictImage(r"image_path", result_count=5)
print(predictions,probabilities)

### Code for saving model to the TensorFlow SavedModel format

In [ ]:
squeezemod = getModelPredict("SqueezeNet",MODEL_DIR+"/model_name",JSON_DIR+"/model_class.json",num_classes)
modelTransfer = squeezemod._CustomImagePrediction__model_collection[0]
modelTransfer.save('/content/drive/MyDrive/saved/modelName')